# K-Nearest

Sirve esencialmente para clasificar valores buscando los puntos de datos “más similares” (por cercanía) aprendidos en la etapa de entrenamiento.
Clasifica cada punto en una categoría, basándose en la categoría de sus vecinos más cercanos. 
Suele utilizarse en sistemas de recomendación, búsqueda semántica y detección de anomalías. 

    https://www.aprendemachinelearning.com/clasificar-con-k-nearest-neighbor-ejemplo-en-python/#:~:text=K%2DNearest%2DNeighbor%20es%20un,el%20mundo%20del%20Aprendizaje%20Autom%C3%A1tico.
    
    Pros: Sencillo de aprender e implementar. Robusto y versátil.
    Contras: Utiliza todo el dataset para entrenar “cada punto” y por eso requiere de uso de mucha memoria y recursos CPU. 
    
    Cómo funciona:
        1. Calcula la distancia entre el item a clasificar y el resto de items del dataset de entrenamiento.
        2. Selecciona los “k” elementos más cercanos (con menor distancia, según la función que se use)
        3. Realiza una “votación de mayoría” entre los k puntos: los de una clase/etiqueta que <<dominen>> decidirán su clasificación final.

In [196]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

df = pd.read_csv("pokemon.csv")

Podemos utilizar este algoritmo para ver cuáles son los Pokémon más defensivos físicamente hablando:

In [197]:
dfP = df[["base_total", "defense", "attack", "sp_defense", "speed", "sp_attack"]].dropna()

Vemos cuántos Pokémon tienen una Defensa de 80 puntos o más

In [198]:
filtro = dfP["defense"] > 80

In [199]:
# Ponemos etiquetas
dfP["defense"][filtro] = "Fuerte"
dfP["defense"][filtro == False] = "Normal-Bajo"

In [200]:
# Todas las columnas que no son la defensa
cols = dfP.drop(["defense"], axis=1)

In [201]:
dfP["defense"].head()

0    Normal-Bajo
1    Normal-Bajo
2         Fuerte
3    Normal-Bajo
4    Normal-Bajo
Name: defense, dtype: object

In [202]:
nbrs_3 = KNeighborsClassifier(n_neighbors = 3, n_jobs = -1)

In [203]:
nbrs_3.fit(cols, dfP["defense"])

KNeighborsClassifier(n_jobs=-1, n_neighbors=3)

In [204]:
pred = nbrs_3.predict(cols)

In [205]:
# El 90% están clasificados correctamente, esta medida puede ser engañosa, ya que no conocemos la proporción de nuestros datos
np.mean(pred == dfP["defense"])

0.9038701622971286

In [206]:
# 32% de Pokemon con una defensa inferior a 80
np.mean(dfP["defense"] == "Fuerte")

0.3233458177278402

In [207]:
# 67% de Pokemon con una defensa superior a 80
np.mean(dfP["defense"] == "Normal-Bajo")

0.6766541822721598

In [208]:
# Vamos a ir ajustando mejor la métrica, miramos solo el vecino de al lado.
nbrs_1 = KNeighborsClassifier(n_neighbors = 1, n_jobs = -1)
nbrs_1.fit(cols, dfP["defense"])
pred2 = nbrs_1.predict(cols)
np.mean(pred2 == dfP["defense"])

1.0

In [209]:
# Exploramos la matriz de confusión para ver cuánto de bien hemos predicho los casos.
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(newdf["defense"], pred2)
print(confusion_matrix)

[[259  57]
 [  0 485]]


In [210]:
# (0,0) -> En este caso el valor real de los datos eran 0 (False = débil) y el modelo ha predicho como 0 (débil).
# (0,1) -> En este caso el valor real de los datos eran 0 (False = défil) y el modelo ha predicho como 1 (fuerte).
# (1,0) -> En este caso el valor real de los datos eran 1 (True = fuerte) y el modelo ha predicho como 0 (débil).
# (1,1) -> En este caso el valor real de los datos eran 1 (True = fuerte) y el modelo ha predicho como 1 (fuerte).